In [ ]:
import psycopg2
import pandas as pd
import numpy as np
import sys
import re
from io import StringIO

In [ ]:
conn = psycopg2.connect( \
    database="thesis",
    user = "postgres", 
    password = "jonp8UMs8qDV4jEcwOC0",
    host = "localhost"
    )
cur = conn.cursor()

In [ ]:
thesis_path = r"C:\Users\cvaka\OneDrive\Master\Thesis"
#thesis_path = r"C:\Users\Christophe\OneDrive\Master\Thesis"
max_entries = 10000
table = 'wfs'

In [ ]:
df_values = pd.read_csv(thesis_path+"/data/weather/WFS_tables_dump/wfs_values_raw.csv")

df_values.rename(columns= {
    "WEATHERFORECASTSCHIPHOL_ID": "id",
    "ROWNUMBER": "row",
    "DATA": "data"
}, inplace=True)

row_dict = {
    1005: "t",
    1006: "f_type", # Forecast type
    2001: "rvr5000_1000",
    2003: "rvr1500_300",
    2004: "rvr550_200",
    2005: "rvr350",
    2011: "rvrcat",
    3001: "wind_dir",
    3002: "wind_dir_std",
    3011: "wind_spd",
    3012: "wind_spd_std",
    3021: "wind_stoten",
    4001: "temp",
    4011: "dew",
    5001: "snow",
    5002: "snow_heavy",
    5011: "rain_cool",
    5021: "cb",
    5031: "lightning",
    6001: "rvr5000_2000"
}

df_values = df_values[df_values["row"].isin(row_dict)]
df_values['row'].replace(row_dict, inplace=True)
df_values['data'] = df_values['data'].str.split(' ')

df_values


In [ ]:
with open(thesis_path+"/data/weather/WFS_tables_dump/wfs_meta_raw.csv", "r") as f:
    df_meta = pd.read_csv(StringIO(re.sub(r'(\d{1,2}\/\d{1,2}\/\d{4}),', r'\1 00:00:00,', f.read())))

df_meta.rename(columns= {
    "WEATHERFORECASTSCHIPHOL_ID": "id",
    "BEGIN_TIME_FORECAST_SHORT": "t_start_short",
    "PUBLICATION_TIME_FORECAST_SHOR": "t_publ_short",
    "BEGIN_TIME_FORECAST_LONG": "t_start_long",
    "PUBLICATION_TIME_FORECAST_LONG": "t_publ_long",
    "VALID_FOR_PERIOD": "fluff"
}, inplace=True)
df_meta = df_meta[df_meta['id'] >= 37882] # Select 2017 onwards, when using the full set the program will fail due to inconsistent reporting of snow in weather IDs 2888 tot 4705
# df_meta = df_meta.astype({
#     "id": int,
#     "t_start_short": "datetime64[ns]",
#     "t_publ_short": "datetime64[ns]",
#     "t_start_long": "datetime64[ns]",
#     "t_publ_long": "datetime64[ns]",
#     })
timecols = ['t_start_short', 't_publ_short', 't_start_long', 't_publ_long']
for col in timecols:
    df_meta[col] = pd.to_datetime(df_meta[col], format= '%d/%m/%Y %H:%M:%S')
df_meta.drop(["fluff"], axis=1, inplace=True)
df_meta.reset_index(drop=True, inplace=True)
df_meta

In [ ]:
data_out = []
failed = []

pd.set_option("mode.chained_assignment", None)

for i, meta in df_meta.iterrows():
    values = df_values[df_values['id']==meta['id']]
    values = values[~values['data'].isna()]
    if len(values) == 0:
        failed.append("No data found for weather ID {}".format(meta['id']))
    elif not 't' in values[['row']].values:
        failed.append("No time data found for weather ID {}".format(meta['id']))
    elif not 'f_type' in values[['row']].values:
        failed.append("No forecast time found for weather ID {}".format(meta['id']))
    else:
        values = pd.DataFrame(np.array(list(values['data'])).transpose(), columns=values['row'])
        values['t_publish'] = pd.to_datetime( \
        (values['f_type']=='k') * ((meta['t_publ_short'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')) + \
        (values['f_type']=='l') * ((meta['t_publ_long'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')), unit='s')
        values['t_test'] = values['t'].copy()
        values['t'][1:] = (values['t'].astype(int)-values['t'].astype(int).shift())[1:] # Calculate difference with previous hour
        values['t'][0] = int(values['t'][0]) - meta['t_start_short'].hour # Calculate hour difference with meta['t_start_short']
        values['t'] = meta['t_start_short'] + pd.to_timedelta((values['t'] + 24*(values['t']<0)).cumsum(), unit='h')

        values.sort_values('t', inplace=True) # Probably not necessary

        values['valid_from'] = pd.to_datetime((values['t'].astype('int64') + values['t'].astype('int64').shift()) / 2)
        values['valid_from'].iloc[0] = values['t'].iloc[0] - pd.Timedelta(30, 'm') * int(values['f_type'].iloc[0]=='k') - pd.Timedelta(90, 'm') * int(values['f_type'].iloc[0]=='l')

        values['valid_till'] = pd.to_datetime((values['t'].astype('int64') + values['t'].astype('int64').shift(-1)) / 2)
        values['valid_till'].iloc[-1] = values['t'].iloc[-1] + pd.Timedelta(30, 'm') * int(values['f_type'].iloc[-1]=='k') + pd.Timedelta(90, 'm') * int(values['f_type'].iloc[-1]=='l')

        values['forecast_id'] = meta['id']
        data_out.append(values)
    if i % int(len(df_meta)/100) == 0:
        pct = int(i/int(len(df_meta)/100))
        sys.stdout.write('\r')
        sys.stdout.write("\t[%-20s] %d%%" % ('='*int(pct/5), pct))
        sys.stdout.flush()

data_out = pd.concat(data_out)

for i in failed:
    print(i)

if len(data_out[data_out['t_test'].astype(int)!= data_out['t'].dt.hour]):
    raise Exception("""Data Inconsistencies for: 
    {}""".format(data_out[data_out['t_test'].astype(int)!= data_out['t'].dt.hour]))

if len(data_out[data_out.duplicated(subset=['forecast_id', 't'], keep=False)]):
    raise Exception("""Primary Key Violation
    {}""".format(data_out[data_out.duplicated(subset=['forecast_id', 't'], keep=False)]))
    
pd.set_option("mode.chained_assignment", "warn")

# 237 s
# values[['t_publish', 'f_type', 't', 'valid_from', 'valid_till']]

In [ ]:
# valid_range
data_out['valid_range'] = '[' + data_out['valid_from'].astype(str) + ', ' + data_out['valid_till'].astype(str) + ')'
# wind_stoten
pd.set_option("mode.chained_assignment", "warn")
data_out['wind_stoten'] = ((data_out['wind_stoten']=="-99").astype(int) * (data_out['wind_spd']) + \
                                (data_out['wind_stoten']!="-99").astype(int) * (data_out['wind_stoten']))

In [ ]:
# Filter & store
data_out = data_out[(data_out[['t','t_publish']] >= "01-01-2005").all(axis=1)]
data_out = data_out.reindex(columns=["forecast_id", 't_publish', 'valid_range']+list(row_dict.values()))
data_out.to_csv("data_out.csv", index=False)

In [ ]:
# query = """
# DROP TABLE IF EXISTS {table};

# CREATE TABLE {table} (
#     forecast_id bigint,
#     t_publish timestamptz,
#     valid_range tstzrange,
#     t timestamptz,
#     f_type text,
#     rvr5000_1000 bigint,
#     rvr1500_300 bigint,
#     rvr550_200 bigint,
#     rvr350 bigint,
#     rvrcat text,
#     wind_dir bigint,
#     wind_dir_std bigint,
#     wind_spd bigint,
#     wind_spd_std bigint,
#     wind_stoten bigint,
#     temp bigint,
#     dew bigint,
#     snow bigint,
#     snow_heavy bigint,
#     rain_cool bigint,
#     cb bigint,
#     lightning bigint,
#     RVR5000_2000 bigint,
#     PRIMARY KEY (forecast_id, t)
# );

# CREATE INDEX {table}_t_publish
#     ON public.{table} USING btree
#     (t_publish ASC NULLS LAST)
# ;

# CREATE INDEX {table}_t
#     ON public.{table} USING btree
#     (t ASC NULLS LAST)
# ;

# CREATE INDEX {table}_valid_range
#     ON public.{table} USING gist
#     (valid_range)
# ;

# CREATE INDEX {table}_forecast_id
#     ON public.{table} USING btree
#     (forecast_id ASC NULLS LAST)
# ;
# """.format(table=table)
# cur.execute(query)
# conn.commit()

In [ ]:
print("""
set PGPASSWORD=jonp8UMs8qDV4jEcwOC0
"C:\\Program Files\\PostgreSQL\\13\\bin\\psql.exe" thesis postgres
\copy public.{table} (forecast_id, t_publish, valid_range, t, f_type, rvr5000_1000, rvr1500_300, rvr550_200, rvr350, rvrcat, wind_dir, wind_dir_std, wind_spd, wind_spd_std, wind_stoten, temp, dew, snow, snow_heavy, rain_cool, cb, lightning, RVR5000_2000) FROM 'C:/Users/cvaka/OneDrive/Master/Thesis/SQL/WEATHER/data_out.csv' DELIMITER ',' CSV HEADER QUOTE '"' ESCAPE '''';
""".format(table=table))

In [ ]:
data_out[data_out['forecast_id']==43423][['forecast_id', 't_publish', 'valid_range', 't']]
# 43423

# DST voor Ferdinand

In [ ]:
with open(thesis_path+"/data/weather/WFS_tables_dump/wfs_meta_raw.csv", "r") as f:
    df_meta = pd.read_csv(StringIO(re.sub(r'(\d{1,2}\/\d{1,2}\/\d{4}),', r'\1 00:00:00,', f.read())))

df_meta.rename(columns= {
    "WEATHERFORECASTSCHIPHOL_ID": "id",
    "BEGIN_TIME_FORECAST_SHORT": "t_start_short",
    "PUBLICATION_TIME_FORECAST_SHOR": "t_publ_short",
    "BEGIN_TIME_FORECAST_LONG": "t_start_long",
    "PUBLICATION_TIME_FORECAST_LONG": "t_publ_long",
    "VALID_FOR_PERIOD": "fluff"
}, inplace=True)
timecols = ['t_start_short', 't_publ_short', 't_start_long', 't_publ_long']
for col in timecols:
    df_meta[col] = pd.to_datetime(df_meta[col], format= '%d/%m/%Y %H:%M:%S')
df_meta.drop(["fluff"], axis=1, inplace=True)
df_meta.reset_index(drop=True, inplace=True)
df_meta

In [ ]:
df_values = pd.read_csv(thesis_path+"/data/weather/WFS_tables_dump/wfs_values_raw.csv")

df_values.rename(columns= {
    "WEATHERFORECASTSCHIPHOL_ID": "id",
    "ROWNUMBER": "row",
    "DATA": "data"
}, inplace=True)

row_dict = {
    1005: "t",
    1006: "f_type", # Forecast type
    2001: "rvr5000_1000",
    2003: "rvr1500_300",
    2004: "rvr550_200",
    2005: "rvr350",
    2011: "rvrcat",
    3001: "wind_dir",
    3002: "wind_dir_std",
    3011: "wind_spd",
    3012: "wind_spd_std",
    3021: "wind_stoten",
    4001: "temp",
    4011: "dew",
    5001: "snow",
    5002: "snow_heavy",
    5011: "rain_cool",
    5021: "cb",
    5031: "lightning",
    6001: "rvr5000_2000"
}

df_values = df_values[df_values["row"].isin(row_dict)]
df_values['row'].replace(row_dict, inplace=True)
df_values['data'] = df_values['data'].str.split(' ')

df_values

In [ ]:
# Limited to values after 2017 for speed
df_meta = df_meta[df_meta['id'] >= 37882]
df_meta.reset_index(drop=True, inplace=True)

In [ ]:
data_out = []
failed = []
for i, meta in df_meta.iterrows():
    values = df_values[df_values['id']==meta['id']]
    values = values[~values['data'].isna()]
    if len(values) == 0:
        failed.append("No data found for weather ID {}".format(meta['id']))
    elif not 't' in values[['row']].values:
        failed.append("No time data found for weather ID {}".format(meta['id']))
    elif not 'f_type' in values[['row']].values:
        failed.append("No forecast time found for weather ID {}".format(meta['id']))
    else:
        values = pd.DataFrame(np.array(list(values['data'])).transpose(), columns=values['row'])
        # values['t_publish'] = pd.to_datetime( \
        # (values['f_type']=='k') * ((meta['t_publ_short'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')) + \
        # (values['f_type']=='l') * ((meta['t_publ_long'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')), unit='s')

        # values['t_test'] = values['t'].copy()

        values['t'][1:] = (values['t'].astype(int)-values['t'].astype(int).shift())[1:] # Calculate difference with previous hour
        values['t'][0] = int(values['t'][0]) - meta['t_start_short'].hour # Calculate hour difference with meta['t_start_short']
        values['t'] = meta['t_start_short'] + pd.to_timedelta((values['t'] + 24*(values['t']<0)).cumsum(), unit='h')

        # values.sort_values('t', inplace=True) # Probably not necessary

        # values['valid_from'] = pd.to_datetime((values['t'].astype('int64') + values['t'].astype('int64').shift()) / 2)
        # values['valid_from'].iloc[0] = values['t'].iloc[0] - pd.Timedelta(30, 'm') * int(values['f_type'].iloc[0]=='k') - pd.Timedelta(90, 'm') * int(values['f_type'].iloc[0]=='l')

        # values['valid_till'] = pd.to_datetime((values['t'].astype('int64') + values['t'].astype('int64').shift(-1)) / 2)
        # values['valid_till'].iloc[-1] = values['t'].iloc[-1] + pd.Timedelta(30, 'm') * int(values['f_type'].iloc[-1]=='k') + pd.Timedelta(90, 'm') * int(values['f_type'].iloc[-1]=='l')


        values['forecast_id'] = meta['id']
        values['t_start_short'] = meta['t_start_short']
        values['t0'] = (values['t'][0])

        data_out.append(values)
    if i % int(len(df_meta)/100) == 0:
        pct = int(i/int(len(df_meta)/100))
        sys.stdout.write('\r')
        sys.stdout.write("\t[%-20s] %d%%" % ('='*int(pct/5), pct))
        sys.stdout.flush()

data_out = pd.concat(data_out)

for i in failed:
    print(i)

# if len(data_out[data_out['t_test'].astype(int)!= data_out['t'].dt.hour]):
#     raise Exception("""Data Inconsistencies for: 
#     {}""".format(data_out[data_out['t_test'].astype(int)!= data_out['t'].dt.hour]))

if len(data_out[data_out.duplicated(subset=['forecast_id', 't'], keep=False)]):
    raise Exception("""Primary Key Violation
    {}""".format(data_out[data_out.duplicated(subset=['forecast_id', 't'], keep=False)]))

# 237 s
# values[['t_publish', 'f_type', 't', 'valid_from', 'valid_till']]

In [ ]:
data_out[(data_out['t_start_short']!=data_out['t0'])][['t', 'forecast_id', 't_start_short', 't0']].drop_duplicates(subset=['forecast_id'], keep='first')

In [ ]:
data_out[(data_out['t_start_short']!=data_out['t0']) & (data_out['t_start_short'].dt.hour==23)][['t', 'forecast_id', 't_start_short', 't0']].drop_duplicates(subset=['forecast_id'], keep='first').sort_values('t')

In [ ]:
data_out[(data_out['t_start_short'].dt.date!=data_out['t0'].dt.date) & (data_out['t_start_short'].dt.hour!=23)].drop_duplicates(subset=['forecast_id'], keep='first')

In [ ]:
values['t'][0] = int(values['t'][0]) - meta['t_start_short'].hour # Calculate hour difference with meta['t_start_short']

values['t'] = meta['t_start_short'] + pd.to_timedelta((values['t'] + 24*(values['t']<0)).cumsum(), unit='h')